### Prob. 5

- Data: seems not limited
- syn method: smartnoise-pacsynth
- Error msg:
    - ValueError: Input contains NaN.

In [1]:
import os
import pprint
import sys

path_petsard = os.path.dirname(os.path.dirname(os.getcwd()))
print(path_petsard)
sys.path.append(path_petsard)

pp = pprint.PrettyPrinter(depth=2)

d:\Dropbox\89_other_application\GitHub\PETsARD


In [2]:
from PETsARD import (
    Loader,
    Processor,
    Synthesizer,
)


load = Loader(filepath='benchmark://adult-income')
load.load()
pp.pprint(load.data.shape)
load.data = load.data.copy().head(3000)
pp.pprint(load.data.shape)


proc = Processor(metadata=load.metadata,)
proc.fit(
    data=load.data,
    sequence=[
        'missing',
        'outlier',
        'scaler',
        'discretizing'
    ],
)
preproc_data = proc.transform(data=load.data,)
pp.pprint(preproc_data.shape)
pp.pprint(preproc_data.head(10))
pp.pprint(preproc_data.isna().sum())

Loader - Benchmarker: file benchmark\adult-income.csv already exist and match SHA-256.
                      PETsARD will ignore download and use local data directly.
(48842, 15)
(3000, 15)
(1824, 15)
   age  workclass  fnlwgt  education  educational-num  marital-status  \
0  0.0          3     0.0          1              0.0               4   
1  0.0          3     0.0         11              0.0               2   
2  0.0          2     0.0          7              0.0               2   
3  0.0          0     0.0         11              0.0               4   
4  0.0          3     0.0         15              0.0               4   
5  0.0          1     0.0          9              0.0               2   
6  0.0          3     0.0         11              0.0               4   
7  0.0          0     0.0         11              0.0               2   
8  0.0          3     0.0         12              0.0               2   
9  0.0          3     0.0         10              0.0               2

In [3]:
syn = Synthesizer(method='smartnoise-pacsynth',)
syn.create(data=preproc_data)
syn.fit_sample()
pp.pprint(syn.data_syn.isna().sum())

Synthesizer (SmartNoise): Fitting pacsynth.
Synthesizer (SmartNoise): Fitting  pacsynth spent 1.3967 sec.
Synthesizer (SmartNoise): Sampling pacsynth # 1824 rows (same as raw) in 14.1106 sec.
age                 49
workclass          371
fnlwgt              94
education          385
educational-num     85
marital-status     256
occupation         546
relationship       384
race               181
gender             143
capital-gain        85
capital-loss       118
hours-per-week      79
native-country     190
income             143
dtype: int64


In [4]:
proc._inverse_sequence

In [5]:
for col in proc._metadata['col'].keys():
    print(f"{col}: {proc._metadata['col'][col]['dtype']}")

age: int8
workclass: category
fnlwgt: int32
education: category
educational-num: int8
marital-status: category
occupation: category
relationship: category
race: category
gender: category
capital-gain: int32
capital-loss: int16
hours-per-week: int8
native-country: category
income: category


In [6]:
proc._config['discretizing']['workclass'].PROC_TYPE == ('encoder', 'discretizing')

True

In [7]:
proc.inverse_transform(data=syn.data_syn)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,371.439451,Private,1.047898e+10,HS-grad,16.666984,Divorced,Adm-clerical,Not-in-family,White,Female,8.470518e+07,120922.973286,173.148118,United-States,<=50K
1,371.439451,Private,1.047898e+10,HS-grad,16.666984,Divorced,Adm-clerical,Not-in-family,White,Female,8.470518e+07,120922.973286,173.148118,United-States,<=50K
2,371.439451,Private,1.047898e+10,HS-grad,16.666984,Divorced,Adm-clerical,Not-in-family,White,Female,8.470518e+07,120922.973286,173.148118,United-States,<=50K
3,371.439451,Private,1.047898e+10,HS-grad,16.666984,Divorced,Adm-clerical,Not-in-family,White,Female,8.470518e+07,120922.973286,173.148118,United-States,<=50K
4,371.439451,Private,1.047898e+10,HS-grad,16.666984,Divorced,Adm-clerical,Not-in-family,White,Female,8.470518e+07,120922.973286,173.148118,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
573,371.439451,Private,1.047898e+10,Bachelors,16.666984,Never-married,Prof-specialty,Not-in-family,White,Male,8.470518e+07,120922.973286,173.148118,United-States,<=50K
574,371.439451,Private,1.047898e+10,Bachelors,16.666984,Never-married,Prof-specialty,Not-in-family,White,Male,8.470518e+07,120922.973286,173.148118,United-States,<=50K
575,371.439451,Private,1.047898e+10,Bachelors,16.666984,Never-married,Prof-specialty,Not-in-family,White,Male,8.470518e+07,120922.973286,173.148118,United-States,<=50K
576,371.439451,Private,1.047898e+10,Bachelors,16.666984,Never-married,Prof-specialty,Not-in-family,White,Male,8.470518e+07,120922.973286,173.148118,United-States,<=50K


In [8]:
print(syn.data_syn.head(6))

   age  workclass  fnlwgt  education  educational-num  marital-status  \
0  0.0        3.0     0.0       11.0              0.0             0.0   
1  0.0        3.0     0.0       11.0              0.0             0.0   
2  0.0        3.0     0.0       11.0              0.0             0.0   
3  0.0        3.0     0.0       11.0              0.0             0.0   
4  0.0        3.0     0.0       11.0              0.0             0.0   
5  0.0        3.0     0.0       11.0              0.0             0.0   

   occupation  relationship  race  gender  capital-gain  capital-loss  \
0        12.0           1.0   4.0     0.0           0.0           0.0   
1        12.0           1.0   4.0     0.0           0.0           0.0   
2        12.0           1.0   4.0     0.0           0.0           0.0   
3        12.0           1.0   4.0     0.0           0.0           0.0   
4        12.0           1.0   4.0     0.0           0.0           0.0   
5        12.0           1.0   4.0     0.0         

age                102
workclass          374
fnlwgt              89
education          418
educational-num     86
marital-status     217
occupation         454
relationship       386
race               180
gender             151
capital-gain       100
capital-loss       107
hours-per-week      99
native-country     184
income             142
dtype: int64
